In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import roc_curve,accuracy_score, roc_auc_score

import statsmodels.api as sm

from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from pygam import LinearGAM,s,f,l

from dmba import classificationSummary

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
Path().resolve() 

WindowsPath('C:/Users/a/git/practical-statistics-for-data-scientists/python/notebooks')

In [5]:
import os
os.chdir('C:/Users/a/git/practical-statistics-for-data-scientists')

In [7]:
DATA=Path().resolve()/'data'
LOAN3000_CSV = DATA / 'loan3000.csv'
LOAN_DATA_CSV = DATA / 'loan_data.csv.gz'
FULL_TRAIN_SET_CSV = DATA / 'full_train_set.csv.gz'

# Naive Bayes

In [8]:
loan_data = pd.read_csv(LOAN_DATA_CSV)

In [11]:
loan_data.head()

,Unnamed: 0,status,loan_amnt,term,annual_inc,dti,payment_inc_ratio,revol_bal,revol_util,purpose,...,delinq_2yrs_zero,pub_rec_zero,open_acc,grade,outcome,emp_length,purpose_,home_,emp_len_,borrower_score
0,1,Charged Off,2500,60 months,30000,1.00,2.39320,1687,9.4,car,...,1,1,3,4.8,default,1,major_purchase,RENT,> 1 Year,0.65
1,2,Charged Off,5600,60 months,40000,5.55,4.57170,5210,32.6,small_business,...,1,1,11,1.4,default,5,small_business,OWN,> 1 Year,0.80
2,3,Charged Off,5375,60 months,15000,18.08,9.71600,9279,36.5,other,...,1,1,2,6.0,default,1,other,RENT,> 1 Year,0.60
3,4,Charged Off,9000,36 months,30000,10.08,12.21520,10452,91.7,debt_consolidation,...,1,1,4,4.2,default,1,debt_consolidation,RENT,> 1 Year,0.50
4,5,Charged Off,10000,36 months,100000,7.06,3.90888,11997,55.5,other,...,1,1,14,5.4,default,4,other,RENT,> 1 Year,0.55


In [20]:
loan_data.outcome=loan_data.outcome.astype('category')
loan_data.outcome.cat.reorder_categories(['paid off', 'default'])
loan_data.purpose_ = loan_data.purpose_.astype('category')
loan_data.home_ = loan_data.home_.astype('category')
loan_data.emp_len_ = loan_data.emp_len_.astype('category')

predictors=['purpose_','home_','emp_len_']
outcome='outcome'
x=pd.get_dummies(loan_data[predictors],prefix='',prefix_sep='')
y=loan_data[outcome]
naive_model= MultinomialNB(alpha=0.01,fit_prior=True)
naive_model = MultinomialNB(alpha=0, fit_prior=False)
naive_model.fit(x,y)

C:\Users\a\anaconda3\lib\site-packages\sklearn\naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


MultinomialNB(alpha=0, fit_prior=False)

In [24]:
new_loan=x.loc[146:146,:]
print('predicted class:\n', naive_model.predict(new_loan)[0])


predicted class:
 default


In [23]:
probabilities=pd.DataFrame(naive_model.predict_proba(new_loan),
                          columns=loan_data[outcome].cat.categories)
print('predicted probabilities: ', probabilities)

predicted probabilities:      default  paid off
0  0.653699  0.346301
